In [1]:
from mssa.mssa import mSSA
import pandas as pd 
import matplotlib.pyplot as plt 
import io
import numpy as np 
import torch 
import copy 
from sklearn.metrics import r2_score
import os 
from datetime import datetime, timedelta
from datetime import timedelta
dev = False

In [2]:
file = "traffic.txt"
isExist = os.path.exists(file)
if not isExist:
    print(f"Files not exist")

In [3]:
"""Preprocess data"""
data = pd.read_csv('traffic.txt', delimiter = ',', header=None)
print('data loaded..')
data_2 = data.copy()
#pick the first 20 clients
data_2 = data_2.iloc[:,:860]
#create time column: 2 years 1 hour
data_2['time'] = pd.to_datetime(np.arange(datetime(2015,1,1), datetime(2017,1,1), timedelta(hours=1)))
data_2.index = data_2['time']
data_2 = data_2.drop(['time'], axis = 1)
#create column names
data_3 = data_2.copy()
col_names = ['MT_{0:03}'.format(i+1) for i in range(data_3.shape[1])]
data_3.columns = col_names
aggregated_data = data_3.copy()

data loaded..


In [4]:
aggregated_data.shape

(17544, 860)

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
Std_normalization = 1
if Std_normalization:
    scaler = StandardScaler()
    temp = scaler.fit_transform(aggregated_data)
    norm_means = scaler.mean_
    norm_std = scaler.scale_
else:
    scaler = MinMaxScaler()
    temp = scaler.fit_transform(aggregated_data)
global data_4
normalized_data = pd.DataFrame(temp, index=aggregated_data.index, columns = aggregated_data.columns)

In [6]:
normalized_data.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_851,MT_852,MT_853,MT_854,MT_855,MT_856,MT_857,MT_858,MT_859,MT_860
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,-0.712761,-0.649143,-0.936348,-1.154807,-0.569033,-0.500971,-0.765328,-0.665566,-0.660995,-0.414196,...,-1.090083,-0.247300,-0.931665,-1.210117,-0.883220,-1.230655,-1.292127,-1.236952,-0.319897,-1.610681
2015-01-01 01:00:00,-0.674341,-0.645010,-0.862140,-1.067719,-0.513278,-0.488945,-0.701394,-0.490022,-0.447094,-0.407787,...,-1.161934,-0.151119,-1.042126,-1.323147,-0.725616,-1.324815,-1.398400,-1.326229,-0.269994,-1.674331
2015-01-01 02:00:00,-0.725568,-0.742124,-0.963111,-1.203794,-0.610849,-0.594173,-0.822705,-0.641794,-0.681695,-0.484698,...,-1.189306,-0.230327,-1.086310,-1.368359,-1.031272,-1.360686,-1.440910,-1.361940,-0.386435,-1.699791
2015-01-01 03:00:00,-0.727168,-0.826841,-1.022720,-1.296324,-0.666604,-0.672342,-0.886640,-0.810024,-0.959996,-0.602201,...,-1.182463,-0.345367,-1.064218,-1.345753,-1.145893,-1.351718,-1.419655,-1.344085,-0.580504,-1.687061
2015-01-01 04:00:00,-0.722366,-0.837172,-1.055566,-1.318096,-0.680543,-0.706917,-0.925984,-0.948996,-1.114097,-0.573359,...,-1.117455,-0.428346,-0.946393,-1.225188,-1.184100,-1.266526,-1.306296,-1.248855,-0.735759,-1.619167


In [7]:
"""

Create missing values based on normal distribution random choice

Input: 
 - pd_data: 2d pandas data frame
 - missing_percentage: missing percentage <= 100

Output:
 - return_data: 2d pandas with missed values

"""
def create_missing_data(pd_data, missing_percentage = 20):
    if missing_percentage == 0: return pd_data
    assert missing_percentage <= 100, "missing percentage should be less than or equal 100%"
    np.random.seed(1993)
    # Convert data frame to array
    np_data = pd_data.to_numpy()
    # Convert original data to 1d array - Because existing package only supports to randomly choose indices from 1d array
    np_data_1d = np_data.flatten()
    # Randomly choose missing index
    total_elem = np_data_1d.shape[0]
    number_of_missing_elem = int(missing_percentage*1.0*total_elem/100)
    missing_index = np.random.choice(np.arange(total_elem), number_of_missing_elem, replace=False) # with replace = False, an index only is chosen 1 time
    # Replace missing_index with 0
    np_data_1d[missing_index] = 0
    # Convert 1d array to 2d array
    np_data_2d = np_data_1d.reshape(pd_data.shape[0], pd_data.shape[1])
    # Convert 2d array to dataframe
    cols_name = pd_data.columns
    return_data = pd.DataFrame(np_data_2d, columns = cols_name)
    return_data.index = pd_data.index
    return return_data

missing_df = create_missing_data(normalized_data, missing_percentage=20)
missing_df.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_851,MT_852,MT_853,MT_854,MT_855,MT_856,MT_857,MT_858,MT_859,MT_860
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,-0.712761,0.000000,-0.936348,-1.154807,-0.569033,-0.500971,-0.765328,0.000000,-0.660995,-0.414196,...,0.000000,-0.247300,0.000000,-1.210117,-0.883220,-1.230655,0.000000,-1.236952,-0.319897,-1.610681
2015-01-01 01:00:00,-0.674341,0.000000,-0.862140,-1.067719,-0.513278,-0.488945,-0.701394,-0.490022,-0.447094,-0.407787,...,-1.161934,-0.151119,-1.042126,0.000000,-0.725616,-1.324815,-1.398400,-1.326229,-0.269994,0.000000
2015-01-01 02:00:00,-0.725568,-0.742124,0.000000,-1.203794,-0.610849,-0.594173,0.000000,0.000000,-0.681695,-0.484698,...,0.000000,-0.230327,0.000000,-1.368359,0.000000,-1.360686,0.000000,-1.361940,-0.386435,-1.699791
2015-01-01 03:00:00,-0.727168,0.000000,-1.022720,-1.296324,-0.666604,-0.672342,-0.886640,-0.810024,-0.959996,-0.602201,...,-1.182463,-0.345367,-1.064218,-1.345753,-1.145893,-1.351718,-1.419655,0.000000,-0.580504,-1.687061
2015-01-01 04:00:00,-0.722366,-0.837172,-1.055566,-1.318096,-0.680543,-0.706917,-0.925984,-0.948996,-1.114097,0.000000,...,-1.117455,-0.428346,0.000000,-1.225188,-1.184100,-1.266526,-1.306296,0.000000,-0.735759,0.000000


In [8]:
global n_clients; global data_train; global data_test
def set_train_test(n_clients=20):
    data_train = normalized_data.iloc[:17376,:n_clients] 
    data_test = normalized_data.iloc[17376:,:n_clients]
    return data_train, data_test

In [9]:
data_train, data_test = set_train_test(n_clients=860)
data_train.shape, data_test.shape

((17376, 860), (168, 860))

In [10]:
def get_flattened_obs(data_train, L=80):
    df = data_train
    N = df.shape[1]
    col_to_row_ratio = 4

    T = df.shape[0]

    M = int(df.size / L)
    if M%N != 0:
        M -= M%N
    M_ts = M // N
    # inc_obs = np.array(df.iloc[:M_ts*L,:]) # first range, we use second range for traning
    inc_obs = np.array(df.iloc[T%L:,:]) # second range, note its not T%L+1 due to python index
    normalize = False
    from sklearn.preprocessing import StandardScaler
    if normalize:
        scaler = StandardScaler()
        inc_obs = scaler.fit_transform(inc_obs)
        norm_means = scaler.mean_
        norm_std = scaler.scale_

    flattened_obs = inc_obs.reshape([L,M], order = 'F') # 按照列顺序
    # flattened_obs = flattened_obs[:,np.arange(M_ts*self.no_ts).reshape([self.no_ts,M_ts]).flatten('F')] # 这里导致第二列是ts2，stacked page是不同ts交错组成
    return flattened_obs, M_ts

In [11]:
global N; global M_ts; global L; global window
L = 100
flattened_obs, M_ts = get_flattened_obs(data_train, L=L)
window = M_ts

In [13]:
M_ts

173

In [12]:
def predict_one_day(data_test, model, weights, days=7):
    # predict for seven days
    # days = 7

    #initialise prediction array
    predictions = np.zeros((len(data_test.columns),24*days))
    ub = np.zeros((len(data_test.columns),24*days))
    lb = np.zeros((len(data_test.columns),24*days))

    # specify start time
    start_time = pd.Timestamp('2016-12-25 00:00:00')

    # actual = data_test.values[:24*days,:]

    # obtain new actual by index, new test start from 2014-12-02-17:00
    actual = data_test[data_test.index>=start_time].values[:24*days,:]


    for day in range(days):
        # get the final time stamp in the day
        end_time = start_time + pd.Timedelta(hours=23)
        # convert timestamps to string
        start_str = start_time.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end_time.strftime('%Y-%m-%d %H:%M:%S')
        # predict for each house
        for i, column in enumerate(data_test.columns):
            # Forecast
            df_30 = model.predict(column,start_str,end_str)
            predictions[i,day*24:(day+1)*24] = df_30['Mean Predictions']
            ub[i,day*24:(day+1)*24] = df_30['Upper Bound']
            lb[i,day*24:(day+1)*24] = df_30['Lower Bound']

        # fit the model with the already predicted values 

        # df_insert = data_test.iloc[day*24:24*(day+1),:]

        # obtain new df_insert
        # df_insert = data_test[data_test.index>=start_time].iloc[day*24:24*(day+1),:]

        # model.update_model(df_insert)
    
        if weights is not None:
            model.ts_model.models[0].weights = weights

        # update start_time
        start_time = start_time + pd.Timedelta(hours=24)
    return actual, predictions

In [15]:
def test_sd(data_train, data_test, L, n_users, M_ts, dim, days, plot_all, plot_single):
    data_train = data_train.iloc[:,:n_users] # Debug1
    data_test = data_test.iloc[:,:n_users]

    flattened_obs, M_ts = get_flattened_obs(data_train, L=L)
    # stand-alone ssa
    window = M_ts
    lst_U_sd = []
    for i in range(n_users):
        data = flattened_obs[:,i*window:(i+1)*window]
        U,_,_ = np.linalg.svd(data)
        U = U[:,:dim]
        lst_U_sd.append(U)

    P_sd = flattened_obs
    P_sd_hat = []
    y_sd = []
    y_true = P_sd[-1,:]
    P_tilde_sd_hat = []
    imputation_model_score_sd = []
    actual = []; predictions_sd = []
    from sklearn.linear_model import LinearRegression
    
    for i in range(n_users):
        P_i_sd = P_sd[:,int(i*window):int((i+1)*window)]
        P_i_sd_hat = lst_U_sd[i].dot(lst_U_sd[i].T.dot(P_i_sd)); P_sd_hat.append(P_i_sd_hat)
        y_i_sd = P_i_sd_hat[-1,:]; y_sd.append(y_i_sd)
        y_i_true = P_i_sd[-1,:]
        P_i_tilde_sd_hat = P_i_sd_hat[:-1,:]; P_tilde_sd_hat.append(P_i_tilde_sd_hat)
        imputation_model_score_sd.append(r2_score(P_i_sd.flatten('F'),P_i_sd_hat.flatten('F'))) # verified same as imputation_model_score)
        # prediction
        reg = LinearRegression(fit_intercept=False).fit(P_i_tilde_sd_hat.T, y_i_sd)
        weights_sd_i = reg.coef_
        model_sd = mSSA(rank = dim, normalize = False, L=L)
        model_sd.update_model(pd.DataFrame(data_train.iloc[:,i]))
        model_sd.ts_model.models[0].weights = weights_sd_i
        actual_i, predictions_sd_i = predict_one_day(pd.DataFrame(data_test.iloc[:,i]), model_sd, weights_sd_i)
        actual.append(actual_i); predictions_sd.append(predictions_sd_i.T)
    imputation_model_score_sd = np.array(imputation_model_score_sd)
    P_sd_hat = np.hstack(P_sd_hat)
    y_sd = np.hstack(y_sd)
    P_tilde_sd_hat = np.hstack(P_tilde_sd_hat)
    actual = np.hstack(actual); predictions_sd = np.hstack(predictions_sd)
    print("imputation score:", imputation_model_score_sd.mean())
    
    Y = actual[:,:]
    Y_h_sd = predictions_sd[:,:]
    mse_sd = np.sqrt(np.mean(np.square(Y[:24*days]-Y_h_sd[:24*days]))) # Debug2
    y_true = Y[:24*days]
    y_pred = Y_h_sd[:24*days]
    print('Forecasting accuracy (RMSE) my:',mse_sd)
    rmse_sd = mse_sd
    if plot_all:
        npar = np.arange(0,20)
    else: npar = [1]
    if plot_single:
        for i in npar:
            plt.figure()
            plt.title('forecasting the next seven days for %s'%data_test.columns[i])
    #         plt.plot(predictions[i,:24*days],label= 'mSSA',color='green')
    #         plt.plot(predictions_my[i,:24*days],label= 'FedmSSA',color='orange')
            plt.plot(predictions_sd.T[i,:24*days],label= 'sd',color='pink')
            plt.plot(actual[:24*days,i],label = 'actual',color='blue')
            plt.legend()
        plt.show()
    
    return Y, Y_h_sd, _, lst_U_sd, rmse_sd, y_true, y_pred

In [16]:
def test_mssa(data_train, data_test, rank, L, n_users, days, plot_all, plot_single):
    data_train = data_train.iloc[:,:n_users]
    data_test = data_test.iloc[:,:n_users]
    model = mSSA(rank = rank, normalize = False, L=L)
    
    # model
    model.update_model(data_train)
    actual, predictions = predict_one_day(data_test, model, None)

    Y = actual[:,:]
    Y_h = predictions.T[:,:]
    mse = np.sqrt(np.mean(np.square(Y[:24*days]-Y_h[:24*days])))
    y_true = Y[:24*days]
    y_pred = Y_h[:24*days]
    print ('Forecasting accuracy (RMSE):',mse)
    rmse_mssa = mse 
    if plot_all:
        npar = np.arange(0,20)
    else: npar = [1]

    if plot_single:
        for i in npar:
            plt.figure()
            plt.title('forecasting the next seven days for %s'%data_test.columns[i])
            plt.plot(predictions[i,:24*days],label= 'mSSA',color='green')
            plt.plot(actual[:24*days,i],label = 'actual',color='blue')
            plt.legend()
        plt.show()
    
    return Y, Y_h, model.ts_model.models[0].weights, rmse_mssa, y_true, y_pred

In [14]:
import copy

def test_fedMssa(data_train, data_test, L, n_users, M_ts, dim, days, plot_all, plot_single, missingVal=0, store=0):    
    data_train = data_train.iloc[:,:n_users]
    data_test = data_test.iloc[:,:n_users]
    flattened_obs, M_ts = get_flattened_obs(data_train, L=L)
    # print(f'M_ts: {M_ts}')
    # model
    model_my = mSSA(rank = dim, normalize = False, L=L)
    # model
    model_my.update_model(data_train)

    # P_admm = flattened_obs

    # ====== 2. read personalized U from h5, which is Ui
    results_path = f"results/SSA/mulTS"
    file_name = f"Traffic860_MissingVal{missingVal}_MulTSTrue_N{n_users}_L100_d100.h5"

    file_path = os.path.join(results_path, file_name)
    lst_U = []

    # Get the list of local U matrix
    import h5py
    with h5py.File(file_path, "r") as f:
        for a_group_key in list(f.keys()):
            ds_arr = f[a_group_key][()]  # returns as a numpy array
            lst_U.append(ds_arr)
    # print(f"lst_U: {len(lst_U)}")

    # Get the list of training data
    P_admm = []
    for i in range(n_users):
        file_name = f'data/data_mulTS/traffic_train_mulTS/traffic_train_nusers_{n_users}_missing_{missingVal}/client_{i}.npy'
        client_i_data = np.load(file_name)
        P_admm.append(client_i_data)

    P_admm = np.array(P_admm)
    # print(P_admm.shape)

    # Select PCs by Sigma
    for i in range(n_users):
        proj_admm_i = lst_U[i].T.dot(P_admm[i])
        S2_admm_i_est = proj_admm_i.dot(proj_admm_i.T)
        S2_admm_i = np.diag(S2_admm_i_est)
        S_admm_i = np.sqrt(S2_admm_i)
        Uk_admm_i = lst_U[i][:,S_admm_i.argsort()[::-1][:dim]] # 针对每一个client，取Uk的那20列，which 取决于 S_admm的大小
        lst_U[i] = Uk_admm_i
    # ============================================================
    
    # Select PCs randomly
#     select_idx = np.random.choice(np.arange(L),dim,replace=False)
#     lst_U = []
#     for i in range(n_users):
#         lst_U.append(Uk_admm[:,:dim])
#     print("Uk shape: ", lst_U[0].shape)

    imputation_model_score_admm = []
    
    P_admm_hat = []
    y_admm = []
    y_true = P_admm[-1,:]
    P_tilde_admm_hat = []
    window = M_ts

    for i in range(n_users):
        P_i_admm = P_admm[i]
        P_i_admm_hat = lst_U[i].dot(lst_U[i].T.dot(P_i_admm)); P_admm_hat.append(P_i_admm_hat)
        y_i_admm = P_i_admm_hat[-1,:]; y_admm.append(y_i_admm)
        y_i_true = P_i_admm[-1,:]
        P_i_tilde_admm_hat = P_i_admm_hat[:-1,:]; P_tilde_admm_hat.append(P_i_tilde_admm_hat)
        imputation_model_score_admm.append(r2_score(P_i_admm.flatten('F'),P_i_admm_hat.flatten('F'))) # verified same as imputation_model_score)
        
        '''Store data into files if non-exist'''
        if store !=0:
            # Store imputed data into files
            x_file_name = f"results/imputed_data/mulTS_traffic/traffic_nusers_{n_users}_missing_{missingVal}/x_client_{i}.npy"
            y_file_name = f"results/imputed_data/mulTS_traffic/traffic_nusers_{n_users}_missing_{missingVal}/y_client_{i}.npy"
            isExist = os.path.exists(x_file_name)
            if not isExist:
                imputed_x = copy.deepcopy(P_i_tilde_admm_hat)
                imputed_y = copy.deepcopy(y_i_admm)
                imputed_x = imputed_x.T
                # print(imputed_x.shape)
                # print(imputed_y.shape)
                np.save(x_file_name, imputed_x)
                np.save(y_file_name, imputed_y)
            else:
                print(f"X and Y Data of client {i} are created")

            '''Store all x, y data into a matrix which facilitates testing step'''
            all_data_file_name = f"results/imputed_data/mulTS_traffic/traffic_nusers_{n_users}_missing_{missingVal}/all_data_client_{i}.npy"
            isExist = os.path.exists(all_data_file_name)
            if not isExist:
                all_data = copy.deepcopy(P_i_admm_hat)
                print(all_data.shape)
                np.save(all_data_file_name, all_data)
            else:
                print(f"All data for client {i} is created ")

    imputation_model_score_admm = np.array(imputation_model_score_admm)

    
    # matrix contain whole data
    imputed_data = P_admm_hat

    # Prepare for fitting model
    
    P_admm_hat = np.hstack(P_admm_hat)
    y_admm = np.hstack(y_admm)
    P_tilde_admm_hat = np.hstack(P_tilde_admm_hat)
    imputed_data = np.array(imputed_data)
    print(f"imputed_data: {imputed_data.shape}")
    print("imputation score", imputation_model_score_admm.mean())

    # verify weights_admm using sklearn
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression(fit_intercept=False).fit(P_tilde_admm_hat.T, y_admm)
    weights_LR = reg.coef_
    
    model_my.ts_model.models[0].weights = weights_LR
    
    actual, predictions_my = predict_one_day(data_test, model_my, weights_LR)

    Y = actual[:,:]
    Y_h_my = predictions_my.T[:,:]
    mse_my = np.sqrt(np.mean(np.square(Y[:24*days]-Y_h_my[:24*days])))
    y_true = Y[:24*days]
    y_pred = Y_h_my[:24*days]
    print ('Forecasting accuracy (RMSE) my:',mse_my)
    rmse_fedmssa = mse_my

    if plot_all:
        npar = np.arange(0,25)
    else: npar = [1]
    if plot_single:
        for i in npar:
            plt.figure()
            plt.title('forecasting the next seven days for %s'%data_test.columns[i])
    #         plt.plot(predictions[i,:24*7],label= 'mSSA',color='green')
            plt.plot(predictions_my[i,:24*days],label= 'FedmSSA',color='orange')
        #     plt.plot(predictions_sd[i,:24*7],label= 'sd',color='pink')
        #     plt.fill_between(np.arange(24*7), lb[i,:24*7], ub[i,:24*7], alpha = 0.1)
            plt.plot(actual[:24*days,i],label = 'actual',color='blue')
            plt.legend()
        plt.show()
    
    return Y, Y_h_my, weights_LR, lst_U, rmse_fedmssa, imputed_data, y_true, y_pred

In [35]:
data_train_missing_vals = create_missing_data(data_train, missing_percentage=80)
data_train_missing_vals.shape

(17376, 860)

In [36]:
data_train_missing_vals.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_851,MT_852,MT_853,MT_854,MT_855,MT_856,MT_857,MT_858,MT_859,MT_860
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,0.000000,0.000000,0.000000,-1.154807,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,-1.236952,-0.319897,0.0
2015-01-01 01:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,-0.488945,0.00000,-0.490022,-0.447094,0.000000,...,-1.161934,0.0,0.0,0.000000,-0.725616,0.0,0.000000,0.000000,0.000000,0.0
2015-01-01 02:00:00,-0.725568,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,-0.484698,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,-0.386435,0.0
2015-01-01 03:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.88664,0.000000,-0.959996,-0.602201,...,0.000000,0.0,0.0,-1.345753,0.000000,0.0,0.000000,0.000000,-0.580504,0.0
2015-01-01 04:00:00,0.000000,-0.837172,-1.055566,0.000000,-0.680543,0.000000,0.00000,-0.948996,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,-1.306296,0.000000,0.000000,0.0


In [19]:
!pwd

/share/home/tung6100/phD/semester3/Federated_MSSA/results/SSA_analysis


In [19]:
%cd 
%cd phD/semester3/Federated_MSSA

/share/home/tung6100
/share/home/tung6100/phD/semester3/Federated_MSSA


In [21]:
create_csv_data = False

In [22]:
if create_csv_data:
    for i in range(data_train_missing_vals.shape[1]):
        data_train_missing_vals.iloc[:,i].to_csv('data/traffic860_train_missing_80/'+ 'MT_{0:03}'.format(i+1) + '.csv')

In [17]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
def evaluate_Testset(y_true, y_pred):
    def mape(y_true, y_pred): 
      return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    MSE = mse(y_true, y_pred)
    RMSE = np.sqrt(MSE)
    MAE = mae(y_true, y_pred)
    MAPE = mape(y_true, y_pred)
    return MSE, RMSE, MAE, MAPE

def accuracy_Testset(y_true, y_pred):
  diff_pred = list()
  diff_true = list()
  accuracy = 0.
  for i in range(len(y_true)-1):
      diff_pred.append(y_pred[i+1]-y_pred[i])
      diff_true.append(y_true[i+1]-y_true[i])
  count = sum(diff_pred[i] * diff_true[i] > 0 for i in range(len(diff_pred)))
  accuracy = count/len(diff_pred) * 100
  return accuracy

# Function to estimate average accuracy for multiple users
def average_acc(y_true, y_pred):
    n_users = y_true.shape[0]
    acc_list = []
    for i in range(n_users):
        y_true_i = y_true[i]
        y_pred_i = y_pred[i]
        acc = accuracy_Testset(y_true_i, y_pred_i)
        acc_list.append(acc)
    acc_np = np.array(acc_list)
    # print(acc_np)
    avg_acc = np.mean(acc_np)
    return avg_acc

In [37]:
rmse_sd = []
for dim in [1,10,20]:
    Y1, Y_sd, weights_sd, lst_U_sd, rmse, y_true_sd, y_pred_sd = test_sd(data_train_missing_vals, data_test, L=L, n_users=860, M_ts=window, dim=dim, days=1, plot_all=False, plot_single=False)
    MSE, RMSE, MAE, MAPE = evaluate_Testset(y_true_sd, y_pred_sd)
    acc = average_acc(y_true_sd, y_pred_sd)
    print(f"MSE: {MSE}, RMSE: {RMSE}, MAE: {MAE}, MAPE: {MAPE}, acc: {acc}")
    
    rmse_sd.append(rmse)
rmse_sd

imputation score: 0.07823488936868085
Forecasting accuracy (RMSE) my: 0.7737014705231056
MSE: 0.598613965489616, RMSE: 0.7737014705231056, MAE: 0.6389807346053114, MAPE: 136.87834905673452, acc: 54.41404734187039
imputation score: 0.38730535926076304
Forecasting accuracy (RMSE) my: 0.7557167373714729
MSE: 0.5711077871433836, RMSE: 0.7557167373714728, MAE: 0.6207409272557095, MAPE: 200.64166413205083, acc: 56.31548311990687
imputation score: 0.574123832009273
Forecasting accuracy (RMSE) my: 2.4959563224028836
MSE: 6.229797963342925, RMSE: 2.495956322402883, MAE: 0.7015196442486717, MAPE: 224.94230660030325, acc: 54.05510283275127


[0.7737014705231056, 0.7557167373714729, 2.4959563224028836]

In [38]:
rmse_mssa = []
for rank in [1,10,20]:
    Y2, Y_mssa, weights_mssa, rmse, y_true_mssa, y_pred_mssa = test_mssa(data_train_missing_vals, data_test, rank=rank, L=L, n_users=860, days=1, plot_all=False, plot_single=False)
    MSE, RMSE, MAE, MAPE = evaluate_Testset(y_true_mssa, y_pred_mssa)
    acc = average_acc(y_true_mssa, y_pred_mssa)
    print(f"MSE: {MSE}, RMSE: {RMSE}, MAE: {MAE}, MAPE: {MAPE}, acc: {acc}")
    rmse_mssa.append(rmse)
rmse_mssa

Forecasting accuracy (RMSE): 0.7467001961461662
MSE: 0.557561182924723, RMSE: 0.7467001961461661, MAE: 0.625200329556292, MAPE: 269.1023988797149, acc: 55.47632906480404
Forecasting accuracy (RMSE): 0.7073582068859238
MSE: 0.5003556328488693, RMSE: 0.7073582068859238, MAE: 0.5645948889406779, MAPE: 210.13688804557532, acc: 56.51920838183935
Forecasting accuracy (RMSE): 4.253893957341898
MSE: 18.095613800309913, RMSE: 4.253893957341898, MAE: 2.5976415311435535, MAPE: 3593.1526484618844, acc: 50.32499029879705


[0.7467001961461662, 0.7073582068859238, 4.253893957341898]

In [37]:
rmse_fedmssa = []
for dim in [1, 10, 20, 30, 40, 50, 60, 70, 80]:
    Y3, Y_my, weights_my, lst_U_my, rmse, imputed_data, y_true, y_pred = test_fedMssa(data_train_missing_vals, data_test, L=L, n_users=10, M_ts=window, dim=dim, days=1, plot_all=False, plot_single=False, missingVal=80, store=0)
    MSE, RMSE, MAE, MAPE = evaluate_Testset(y_true, y_pred)
    acc = average_acc(y_true, y_pred)
    print(f"MSE: {MSE}, RMSE: {RMSE}, MAE: {MAE}, MAPE: {MAPE}, acc: {acc}")
    rmse_fedmssa.append(rmse)
rmse_fedmssa

imputed_data: (10, 100, 14878)
imputation score 0.018483733262548395
Forecasting accuracy (RMSE) my: 0.7280956033321795
MSE: 0.5301232075916504, RMSE: 0.7280956033321795, MAE: 0.5905299296962374, MAPE: 186.7975270445052, acc: 56.01851851851851
imputed_data: (10, 100, 14878)
imputation score 0.13299954341705583
Forecasting accuracy (RMSE) my: 529.6430814030541
MSE: 280521.7936781222, RMSE: 529.6430814030541, MAE: 238.4468688839957, MAPE: 133891.29592279065, acc: 51.85185185185185
imputed_data: (10, 100, 14878)
imputation score 0.24195894540372195
Forecasting accuracy (RMSE) my: 0.6812388743767446
MSE: 0.464086403962094, RMSE: 0.6812388743767446, MAE: 0.5598033304765119, MAPE: 219.89991605707698, acc: 51.38888888888889
imputed_data: (10, 100, 14878)
imputation score 0.3448216686195894
Forecasting accuracy (RMSE) my: 0.6136123359496511
MSE: 0.3765200988295876, RMSE: 0.6136123359496511, MAE: 0.5066333576090208, MAPE: 216.92802045843592, acc: 56.944444444444436
imputed_data: (10, 100, 14878

[0.7280956033321795,
 529.6430814030541,
 0.6812388743767446,
 0.6136123359496511,
 0.5813165143573312,
 0.5654810735170337,
 0.5639389684923395,
 0.5685071294488299,
 0.5688770523918311]

In [121]:
# Get imputed data from the FedMSSA with the best rank (=30 for example)
store = 1
Y3, Y_my, weights_my, lst_U_my, rmse, imputed_data = test_fedMssa(data_train_missing_vals, data_test, L=L, n_users=430, M_ts=window, dim=60, days=1, plot_all=False, plot_single=False, missingVal=80, store=store)

imputed_data.shape


(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)
(100, 346)

(430, 100, 346)